# Test herdnet CBAM

## Installations

In [1]:
# Check GPU
!nvidia-smi

Fri May 16 14:51:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# Download and install the code
import sys

!git clone https://github.com/victordpd19/HerdNet
!cd '/content/HerdNet' && python setup.py install

sys.path.append('/content/HerdNet')

Cloning into 'HerdNet'...
remote: Enumerating objects: 592, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 592 (delta 149), reused 140 (delta 132), pack-reused 387 (from 3)
Receiving objects: 100% (592/592), 410.08 KiB | 9.32 MiB/s, done.
Resolving deltas: 100% (377/377), done.
running install
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.11/dist-packages/setuptools/_di

## Create datasets

In [10]:
#  some of the data of Delplanque et al. (2021) as an example
# https://drive.google.com/file/d/1KfGBYaXo9LVOiz9qO1pxRH3ZC5bsywCW/view?usp=drive_link
# !gdown 1CcTAZZJdwrBfCPJtVH6VBU3luGKIN9st -O /content/data.zip # this is the original
!gdown 1KfGBYaXo9LVOiz9qO1pxRH3ZC5bsywCW -O /content/data.zip # this is a backup, same data zip
!unzip -oq /content/data.zip -d /content

Downloading...
From (original): https://drive.google.com/uc?id=1KfGBYaXo9LVOiz9qO1pxRH3ZC5bsywCW
From (redirected): https://drive.google.com/uc?id=1KfGBYaXo9LVOiz9qO1pxRH3ZC5bsywCW&confirm=t&uuid=b4670336-e66c-4fb9-ad2a-5d9da059f425
To: /content/data.zip
100% 11.6G/11.6G [03:32<00:00, 54.5MB/s]


In [5]:
# Set the seed
from animaloc.utils.seed import set_seed

set_seed(9292)

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [6]:
# Download cbam model
!gdown 1ri_1Nwvf72Bm1sh4FI3DMox2HoaCtR7H -O /content/cbam_model.pth


Downloading...
From (original): https://drive.google.com/uc?id=1ri_1Nwvf72Bm1sh4FI3DMox2HoaCtR7H
From (redirected): https://drive.google.com/uc?id=1ri_1Nwvf72Bm1sh4FI3DMox2HoaCtR7H&confirm=t&uuid=dd9a8dfa-0ffd-4ea9-8542-406065874001
To: /content/cbam_model.pth
100% 75.1M/75.1M [00:02<00:00, 35.4MB/s]


In [7]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint = torch.load("cbam_model.pth", map_location=device, weights_only=False)
classes = checkpoint['classes']
num_classes = len(classes) + 1
img_mean = checkpoint['mean']
img_std = checkpoint['std']

In [8]:
model= checkpoint['model']

In [11]:
# Training, validation and test datasets
import albumentations as A
from animaloc.datasets import CSVDataset
from animaloc.data.transforms import DownSample

patch_size = 512
num_classes = 7
down_ratio = 2

albu_transforms = [A.Normalize(mean=img_mean, std=img_std)]

test_dataset = CSVDataset(
    csv_file = '/content/data/test.csv',
    root_dir = '/content/data/test',
    albu_transforms = albu_transforms,
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )



In [12]:
# Dataloaders
from torch.utils.data import DataLoader

test_dataloader = DataLoader(dataset = test_dataset, batch_size = 1, shuffle = False)

## Create the Evaluator

In [13]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

work_dir = '/content/output'
mkdir(work_dir)

test_dir = '/content/test_output'
mkdir(test_dir)

metrics = PointsMetrics(radius=5, num_classes=num_classes) # radious 20

stitcher = HerdNetStitcher(
    model=model,
    size=(patch_size,patch_size),
    overlap=160,
    down_ratio=down_ratio,
    reduction='mean',
    device_name = device
    )

# Create an Evaluator
test_evaluator = HerdNetEvaluator(
    model=model,
    dataloader=test_dataloader,
    metrics=metrics,
    lmds_kwargs = dict(kernel_size=(3,3), adapt_ts=0.2, neg_ts=0.1), #lmds: local maximum detection strategy
    device_name = device,
    stitcher=stitcher,
    work_dir=test_dir,
    header='test'
    )

In [14]:
import wandb  # Set up wandb
from datetime import datetime

wandb.init(
    project = "colab_cbam_run",
    entity = "victorperezd19-universidad-de-los-andes",
    config = dict(
        model = model,
        down_ratio = down_ratio,
        num_classes = num_classes,
        threshold = 5 # change
        )
    )

date = datetime.now().strftime('%Y-%m-%d')
myrun = "myrun"
wandb.run.name = f'{date}_' + myrun + f'_RUN_{wandb.run.id}'

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: victorperezd19 (victorperezd19-universidad-de-los-andes) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [15]:
test_f1_score = test_evaluator.evaluate(returns='f1_score')

/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
INFO:20250516_evaluation:test [  1/258] eta: 0:41:23 n: 1 recall: 1.0 precision: 0.12 f1-score: 0.22 MAE: 7.0 MSE: 49.0 RMSE: 7.0 time: 9.6257 data: 1.6895 max mem: 1471


test [  1/258] eta: 0:41:23 n: 1 recall: 1.0 precision: 0.12 f1-score: 0.22 MAE: 7.0 MSE: 49.0 RMSE: 7.0 time: 9.6257 data: 1.6895 max mem: 1471


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

test [ 11/258] eta: 0:32:00 n: 1 recall: 0.0 precision: 0.0 f1-score: 0.0 MAE: 1.0 MSE: 1.0 RMSE: 1.0 time: 7.7428 data: 1.2272 max mem: 1472


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

...

test [241/258] eta: 0:02:08 n: 13 recall: 0.54 precision: 0.7 f1-score: 0.61 MAE: 3.0 MSE: 9.0 RMSE: 3.0 time: 6.3148 data: 0.9603 max mem: 1472


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

test [251/258] eta: 0:00:56 n: 2 recall: 1.0 precision: 0.29 f1-score: 0.44 MAE: 5.0 MSE: 25.0 RMSE: 5.0 time: 6.4106 data: 0.9672 max mem: 1472


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

test [258/258] eta: 0:00:07 n: 3 recall: 1.0 precision: 0.27 f1-score: 0.43 MAE: 8.0 MSE: 64.0 RMSE: 8.0 time: 6.3364 data: 0.9157 max mem: 1472
test Total time: 0:30:27 (7.0838 s / it)


In [16]:
detections = test_evaluator.results
detections

,class,n,recall,precision,f1_score,confusion,mae,mse,rmse,ap
0,1,675,0.856296,0.698068,0.769128,0.041459,2.104762,10.009524,3.163783,0.763495
1,2,349,0.641834,0.729642,0.682927,0.135135,3.000000,26.666667,5.163978,0.542759
2,3,477,0.857442,0.765918,0.809100,0.131635,1.191667,3.875000,1.968502,0.813615
3,4,74,0.337838,0.165563,0.222222,0.074074,2.409091,10.257576,3.202745,0.185801
4,5,36,0.750000,0.385714,0.509434,0.068966,1.517241,3.379310,1.838290,0.590199
5,6,688,0.709302,0.288076,0.409740,0.000000,8.592000,281.744000,16.785232,0.514607
6,binary,2299,0.816442,0.523717,0.638110,0.000000,5.686047,148.391473,12.181604,0.639674


In [18]:
import pandas
cm = pandas.DataFrame(metrics.confusion_matrix)
print(cm)

       0      1      2     3     4      5
0  578.0    0.0    7.0   8.0  10.0    0.0
1    0.0  224.0    0.0  30.0   4.0    1.0
2   59.0    0.0  409.0   1.0   2.0    0.0
3    0.0    0.0    0.0  25.0   1.0    1.0
4    1.0    0.0    1.0   0.0  27.0    0.0
5    0.0    0.0    0.0   0.0   0.0  488.0
